# Futures

In [1]:
%run -m literary.notebook

In [2]:
import asyncio
import weakref

It can be convenient to create new futures for the running loop. Here we'll implement a simple helper function.

In [3]:
def create_future() -> asyncio.Future:
    """Create a future for the running loop
    
    :returns: new future
    """
    loop = asyncio.get_running_loop()
    return loop.create_future()

There are many ways that futures can be chained together. This helper class makes it easy to select which events to handle.

In [4]:
class FutureChain:
    """Helper class to chain asyncio futures"""
    
    def on_destination_cancelled(self, source, destination):
        source.cancel()

    def on_source_cancelled(self, source, destination):
        destination.cancel()

    def on_source_exception(self, source, destination):
        destination.set_exception(source.exception())

    def on_source_result(self, source, destination):
        destination.set_result(source.result())

    def __call__(self, source, destination):
        assert isinstance(source, asyncio.Future)
        assert isinstance(destination, asyncio.Future)

        @destination.add_done_callback
        def on_destination_done(destination, source_ref=weakref.ref(source)):
            if (source := source_ref()) and destination.cancelled():
                self.on_destination_cancelled(source, destination)

        @source.add_done_callback
        def on_source_done(source, destination_ref=weakref.ref(destination)):
            destination = destination_ref()
            if not destination or destination.cancelled():
                return

            if source.cancelled():
                self.on_source_cancelled(source, destination)

            elif source.exception() is not None:
                self.on_source_exception(source, destination)

            else:
                self.on_source_result(source, destination)

Here we implement some common configurations:

In [5]:
chain_future = FutureChain()

In [6]:
class _FutureChainException(FutureChain):
    def on_source_result(self, source, destination):
        return

In [7]:
chain_future_exception = _FutureChainException()

In [8]:
class _FutureChainHandle(FutureChain):
    def on_source_result(self, source, destination):
        destination.set_result(True)

In [9]:
chain_future_handle = _FutureChainHandle()